In [ ]:
!pip install keras-ctcmodel

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, Concatenate, GRU, Conv2D, MaxPool2D, MaxPooling2D, Dropout, Reshape, add, concatenate, Lambda, LSTM
import keras.backend as K
from keras.models import Model
# from keras_ctcmodel.CTCModel import CTCModel
from scipy import ndarray
import cv2

In [9]:
!unzip '/content/drive/MyDrive/Colab Datasets/handwritten text detection.zip'

unzip:  cannot find or open /content/self_lines, /content/self_lines.zip or /content/self_lines.ZIP.


In [13]:
images_dir = '/content/self_lines'
lines_path = '/content/lines.txt'

In [14]:
img_file_name_list = []
orginal_text_list = []

def img_files_to_org_text():
    loaded_text = open(lines_path, 'r').read()
    loaded_text = loaded_text.strip().split('\n')
    for line in loaded_text:
        img_file_name = line.split(' ')[0] + '.png'
        org_text = line.split(' ')[9].split('|')
        org_text = ' '.join(org_text)
        img_file_name_list.append(img_file_name)
        orginal_text_list.append(org_text)
    return img_file_name_list, orginal_text_list

In [15]:
img_file_name, orginal_text = img_files_to_org_text()

In [8]:
# img_height = 50
# img_width = 400

In [20]:
def random_noise(image: ndarray):
    row,col= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.3
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = image + gauss
    return noisy

def blur_filter(image: ndarray):
  return cv2.blur(image, (4, 4))

def reduce_line_thickness(image: ndarray):
  kernel = np.ones((2, 2), np.uint8)
  return cv2.dilate(image, kernel, iterations = 1)

def random_stretch(image: ndarray):
  stretch = 0.25  # -0.5 .. +0.5
  wStretched = max(int(image.shape[1] * (1 + stretch)), 1)  # random width, but at least 1
  img = cv2.resize(image, (wStretched, image.shape[0]))  # stretch horizontally by factor 0.5 .. 1.5
  return img

In [21]:
img_w = 400
# Input Parameters
img_h = 64

In [28]:
def load_all_images_tensor(img_file_name, orginal_text):
    
    # 1. Read image
    img = tf.io.read_file(img_file_name)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_h, img_w])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])

    return img, orginal_text

In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((img_file_name, orginal_text))

train_dataset = train_dataset.map(load_all_images_tensor, num_parallel_calls = tf.data.experimental.AUTOTUNE
                                  ).batch(32).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

In [24]:
all_images = []
all_texts = []

def load_all_images(img_file_name, orginal_text):
    for file, text in zip(img_file_name, orginal_text):
        
        image = cv2.imread(os.path.join(images_dir, file), cv2.IMREAD_GRAYSCALE)
        (h, w) = image.shape
        final_img = np.ones([64, 400])*255 # blank white image
        if w > 400:
            image = image[:, :400]
        if h > 64:
            image = image[:64, :]
            
        final_img[:h, :w] = image
        image = cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)
        image = image/255
        
        all_images.append(image.reshape(image.shape[0], image.shape[1], 1))
        all_texts.append(text)
        
        img_blur = blur_filter(image)
        all_images.append(img_blur.reshape(img_blur.shape[0], img_blur.shape[1], 1))
        all_texts.append(text)
        
        img_reduce_thick = reduce_line_thickness(image)
        all_images.append(img_blur.reshape(img_blur.shape[0], img_blur.shape[1], 1))
        all_texts.append(text)
        
    return all_images, all_texts

In [30]:
final_images, final_texts = load_all_images(img_file_name, orginal_text)

In [31]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level = True, lower = False)
tokenizer.fit_on_texts(final_texts)
label_seq = tokenizer.texts_to_sequences(final_texts)
label_pad = tf.keras.preprocessing.sequence.pad_sequences(label_seq, padding = 'post')

In [32]:
vocab_size = len(tokenizer.word_index) + 1

In [33]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [35]:
len(final_images), len(final_texts)

(1008, 1008)

In [67]:
# Network parameters
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 512
minibatch_size = 32
unique_tokens = 28
act = 'relu'

input_data = Input(name = 'the_input', shape = (img_w, img_h, 1), dtype='float32')
inner = Conv2D(conv_filters, kernel_size, padding = 'same',
               activation = act, kernel_initializer = 'he_normal',
               name = 'conv1')(input_data)
inner = MaxPooling2D(pool_size = (pool_size, pool_size), name = 'max1')(inner)
inner = Conv2D(conv_filters, kernel_size, padding = 'same',
               activation = act, kernel_initializer = 'he_normal',
               name = 'conv2')(inner)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name = 'max2')(inner)

conv_to_rnn_dims = (img_w // (pool_size ** 2), (img_h // (pool_size ** 2)) * conv_filters)
inner = Reshape(target_shape = conv_to_rnn_dims, name = 'reshape')(inner)

# cuts down input size going into RNN:
inner = Dense(time_dense_size, activation = act, name = 'dense1')(inner)

# Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = GRU(rnn_size, return_sequences = True, kernel_initializer = 'he_normal', name = 'gru1')(inner)
gru_1b = GRU(rnn_size, return_sequences = True, kernel_initializer = 'he_normal', go_backwards = True, name = 'gru1_b')(inner)
gru1_merged = add([gru_1, gru_1b])
gru_2 = GRU(rnn_size, return_sequences = True, kernel_initializer = 'he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences = True, kernel_initializer = 'he_normal', go_backwards = True, name = 'gru2_b')(gru1_merged)

# transforms RNN output to character activations:
y_pred = Dense(vocab_size + 1, name='dense2', activation = 'softmax')(concatenate([gru_2, gru_2b]))

cnn_model = Model(inputs = input_data, outputs = y_pred)

labels = Input(name='the_labels', shape=[label_pad.shape[1]], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
# Keras doesn't currently support loss funcs with extra parameters
# so CTC loss is implemented in a lambda layer

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

# clipnorm seems to speeds up convergence
sgd = tf.keras.optimizers.Adam(lr = 0.002)

model = Model(inputs = [input_data, labels, input_length, label_length], outputs = loss_out)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = sgd, metrics = ['accuracy'])

tf.keras.utils.plot_model(
    model, to_file='/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/htr_model_lstm.png', show_shapes = True, show_dtype = True, show_layer_names = True,  dpi=400
)

In [68]:
input_length = np.ones([len(final_images), 1]) * (100 - 2)
label_length = np.zeros([len(final_images), 1])
train_output = np.zeros([len(final_images)])

for i in range(len(final_images)):
    label_length[i] = len(label_seq[i])

In [ ]:
model.fit([np.array(final_images), label_pad, input_length, label_length], train_output, batch_size = 16, epochs = 50)

In [19]:
model.save('/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/normal_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/normal_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/normal_model/assets


In [71]:
# serialize model to JSON
gru_model_json = model.to_json()
with open("/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/gru_model.json", "w") as json_file:
    json_file.write(gru_model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/gru_model_weights.h5")

In [72]:
json_file = open('/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/gru_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/DL & ML Models/Handwritten_text_detection/gru_model_weights.h5")

In [ ]:
plt.figure(figsize=(15, 10))
img_dir = '/content/a02-004-10.png'
image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
plt.imshow(image, cmap = 'gray')



(h, w) = image.shape
final_img = np.ones([64, 400])*255 # blank white image
if w > 400:
    image = image[:, :400]
if h > 64:
    image = image[:64, :]
    
final_img[:h, :w] = image
image = cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)
image = image/255.

pred = cnn_model.predict(image.reshape(1, 400, 64, 1))
decoded = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], 
                                    greedy = True)[0][0])
print(decoded)
    
plt.subplots_adjust(wspace=0.2, hspace=-0.8)

In [80]:
pred_seq = tokenizer.sequences_to_texts(decoded)

In [ ]:
pred_seq

## Future improvements

###### 1) Implementing HTR using MDLSTM. (This approach can read the handwritten text written in multiple lines or sentences)

###### links: https://arxiv.org/pdf/0705.2011.pdf
######        https://arxiv.org/pdf/1604.03286.pdf
              

###### 2) HTR using transformers. (For better accuracy)

###### links: https://arxiv.org/pdf/2005.13044.pdf

###### 3) Increase the training data size